# Data Cleaning

All the modifications to the data set except for type conversion and column
renames go here

To do: If you end up adding pdays to the model, you need to do better null handling than setting to 999's


In [1]:
%load_ext sql

In [2]:
%sql postgresql://localhost/bankcalls

'Connected: @bankcalls'

# Feature engineering plan

1. Campagin -> Turn everything 7 or more to a 6 and have a category for 6 or more
2. Previous -> before this campaign, for this client
    Bucket this as 3 or more is 3
    
(See EDA output)

In [3]:
%sql UPDATE bank_addl SET campaign = 6 WHERE campaign > 6; 

 * postgresql://localhost/bankcalls
2406 rows affected.


[]

In [4]:
%sql UPDATE bank_addl SET previous = 3 WHERE previous > 3;

 * postgresql://localhost/bankcalls
94 rows affected.


[]

In [5]:
%sql COMMIT;

 * postgresql://localhost/bankcalls
Done.


[]

# Mark the holdout set

In [4]:
%config SqlMagic.autopandas = True

In [5]:
from sklearn.model_selection import train_test_split

df = %sql SELECT * from bank_addl;

y = df['success'].replace({'yes':1, 'no':0})
X = df.drop(columns='success')

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                         test_size=0.2, 
                                         stratify=y,
                                         random_state=3)

In [15]:
%sql ALTER TABLE bank_addl ADD COLUMN test BOOLEAN DEFAULT FALSE;

 * postgresql://localhost/bankcalls
Done.


""


In [16]:
%%capture
for i in X_test.bank_addl_id:
    %sql UPDATE bank_addl SET test = TRUE WHERE bank_addl_id = $i;

In [17]:
%%sql
SELECT COUNT(test) FROM bank_addl GROUP BY test;

 * postgresql://localhost/bankcalls
2 rows affected.


,count
0,32950
1,8238


In [18]:
len(y_train)

32950

In [19]:
len(y_test)

8238

In [20]:
%sql CREATE VIEW test_view AS SELECT * FROM bank_addl WHERE test = TRUE;
%sql CREATE VIEW train_view AS SELECT * FROM bank_addl WHERE test = FALSE;
%sql COMMIT;

 * postgresql://localhost/bankcalls
Done.
 * postgresql://localhost/bankcalls
Done.
 * postgresql://localhost/bankcalls
Done.


""
